In [1]:
import pyspark

In [3]:
conf = pyspark.SparkConf()\
.setMaster("spark://hadoop-master:7077")\
.setAppName("test Spark")

In [4]:
sc = pyspark.SparkContext(conf=conf)

In [2]:
sc.stop()

In [2]:
data = sc.textFile("hdfs://hadoop-master:9999/data.csv")
header = data.first()
print(header)

jobid,keyword,title,place,salary,experience,education,companytype,company_name,jobwelf,time


In [3]:
data2 = data.filter(lambda row : row != header)
data_fields = data2.map(lambda line : line.split(",")).filter(lambda row : len(row)==11)

[['132607755',
  'C++',
  '采销经理（3C）',
  '北京',
  '1-1.5万/月',
  '3-4年经验',
  '大专',
  '创业公司',
  '北京华创盈讯科技有限公司',
  '五险一金 弹性工作 通讯补贴 餐饮补贴 年终奖金 13薪 定期体检 员工旅游 绩效奖金',
  '06-08']]

In [44]:
dataJson = data_fields.map(lambda x:{"jobid":x[0],'keyword':x[1],'title':x[2],'place':x[3],'salary':x[4],'experience':x[5],'education':x[6],'companytype':x[7],'company_name':x[8],'jobwelf':x[9],'time':x[10]})
open('/home/hadoop/Desktop/data/dataJson','w').write(str(dataJson.collect()))

24370541

In [43]:
cityCompanytype = data_fields.map(lambda x : ((x[7],x[3]),1)).reduceByKey(lambda x,y : x+y)
cityCompanytype = cityCompanytype.filter(lambda x : not x[0][0] == '')
cityCompanytype = cityCompanytype.map(lambda x:{"city":x[0][1],'companytype':x[0][0],"value":x[1]})
open('/home/hadoop/Desktop/data/cityCompanytype.json','w').write(str(cityCompanytype.collect()))

34744

In [8]:
# 数据总量
data_fields.count()+1

94647

In [9]:
# 企业个数
companysNum = data_fields.map(lambda x : (x[8],1)).reduceByKey(lambda x,y : x+y)
companysNum.count()+1

9957

In [52]:
companysNum = data_fields.map(lambda x : (x[7],1)).reduceByKey(lambda x,y : x+y)
companysNum.collect()

[('创业公司', 2230),
 ('上市公司', 11191),
 ('民营公司', 71120),
 ('外资（欧美）', 1446),
 ('合资', 3197),
 ('外资（非欧美）', 1730),
 ('', 22),
 ('国企', 3179),
 ('事业单位', 420),
 ('非营利组织', 77),
 ('政府机关', 26),
 ('外企代表处', 8)]

In [28]:
# 各城市岗位数量
cityJobNum = data_fields.map(lambda x : (x[3],1)).reduceByKey(lambda x,y : x+y)
cityJobNum = cityJobNum.map(lambda x:{"city":x[0],"value":x[1]})
open('/home/hadoop/Desktop/data/cityJobNum.json','w').write(str(cityJobNum.collect()))

6234

In [20]:
# combineByKey参数集
createCombiner = (lambda value: (value,1))
mergeValue = (lambda acc,value: (acc[0]+value,acc[1]+1)) 
mergeCombiners = (lambda acc1, acc2 : (acc1[0]+acc2[0],acc1[1]+acc2[1]))

In [30]:
# 不同经验(年)学历的平均工资(万)
import re
import numpy as np
get_unit = lambda x : 10000 if ('万' in x) else(1000 if ('千' in x) else 1)
get_time = lambda x : 12 if ('年' in x) else 1
get_mean = lambda x: (np.mean(x.astype(np.float64)))
get_number = lambda x : np.array(re.findall(r"\d+\.?\d*",x))
ExperienceDegreeSalary = data_fields.map(lambda x : (x[4],x[5],x[6]))
ExperienceDegreeSalary = ExperienceDegreeSalary.map(lambda x : (round(get_mean(get_number(x[0]).astype(np.float64) * get_unit(x[0])/get_time(x[0]))/10000,1),x[1], x[2]))
ExperienceDegreeSalary = ExperienceDegreeSalary.filter(lambda x : not np.isnan(x[0]) and x[2] in ['高中','本科','大专','中专','博士','硕士'])
other = ExperienceDegreeSalary.filter(lambda x : x[1] in '无需经验 在校生/应届生')
ExperienceDegreeSalary = ExperienceDegreeSalary.filter(lambda x : x[1] not in '无需经验 在校生/应届生')
ExperienceDegreeSalary = ExperienceDegreeSalary.map(lambda x : (x[0],int(get_mean(get_number(x[1]).astype(np.float64))), x[2]))
ExperienceDegreeSalary = ExperienceDegreeSalary.filter(lambda x : not np.isnan(x[1]))
ExperienceDegreeSalary  = ExperienceDegreeSalary.union(other)

ExperienceDegreeSalary = ExperienceDegreeSalary.map(lambda x:((x[1],x[2]),x[0]))
ExperienceDegreeSalary = ExperienceDegreeSalary.combineByKey(createCombiner,mergeValue,mergeCombiners)
ExperienceDegreeSalary = ExperienceDegreeSalary.map(lambda x:(x[0],round(float(x[1][0])/x[1][1],1)))
ExperienceDegreeSalary = ExperienceDegreeSalary.map(lambda x:(x[0][0],x[0][1],x[1]))
ExperienceDegreeSalary = ExperienceDegreeSalary.map(lambda x:{"experience":x[0],"degree":x[1],"salary":x[2]})
open('/home/hadoop/Desktop/data/ExperienceDegreeSalary.json','w').write(str(ExperienceDegreeSalary.collect()))

2181

In [32]:
# 词云
import jieba
get_seg = lambda x : (jieba.lcut(x, cut_all=False))
description = data_fields.flatMap(lambda x : get_seg(x[9]))
description = description.map(lambda x : (x,1))
description = description.reduceByKey(lambda x,y : x+y).sortBy(lambda x:x[1],False)
isChinese = lambda c : '\u4e00' <= c <= '\u9fa5'
description = description.filter(lambda x : all(map(isChinese,x[0])))
description = description.filter(lambda x : len(x[0]) >= 2)
description.take(1)
description = description.map(lambda x:{"description":x[0],"value":x[1]})
open('/home/hadoop/Desktop/data/description.json','w').write(str(description.collect()))

42126

In [34]:
# 统计不同日期平均薪资
import re
import numpy as np
get_unit = lambda x : 10000 if ('万' in x) else(1000 if ('千' in x) else 1)
get_time = lambda x : 12 if ('年' in x) else 1
get_mean = lambda x,y : (np.mean(x.astype(np.float64) * get_unit(y)/get_time(y)))
get_number = lambda x : np.array(re.findall(r"\d+\.?\d*",x))
daySalary = data_fields.map(lambda x : (x[4],x[10]))
daySalary = daySalary.map(lambda x : (x[1], get_mean(get_number(x[0]).astype(np.float64),x[0])))
daySalary = daySalary.filter(lambda x : not np.isnan(x[1]))
daySalary_sum = daySalary.combineByKey(createCombiner,mergeValue,mergeCombiners)
daySalary_mean = daySalary_sum.map(lambda x:(x[0],float(x[1][0])/x[1][1]/10000,x[1][1]))
daySalary_mean = daySalary_mean.map(lambda x: (x[0],round((x[1]),1),x[2])).sortBy(lambda x : x[0])
daySalary_mean.take(1)
daySalary_mean = daySalary_mean.map(lambda x:{"day":x[0],"salary":x[1],"value":x[2]})
open('/home/hadoop/Desktop/data/daySalary.json','w').write(str(daySalary_mean.collect()))

2964

In [36]:
# 各城市平均工资
import re
import numpy as np
get_unit = lambda x : 10000 if ('万' in x) else(1000 if ('千' in x) else 1)
get_time = lambda x : 12 if ('年' in x) else 1
get_mean = lambda x,y : (np.mean(x.astype(np.float64) * get_unit(y)/get_time(y)))
get_number = lambda x : np.array(re.findall(r"\d+\.?\d*",x))

citySalary = data_fields.map(lambda x : (x[3],x[4]))
citySalary = citySalary.map(lambda x : (x[0], get_mean(get_number(x[1]).astype(np.float64),x[1])))
citySalary = citySalary.filter(lambda x : not np.isnan(x[1]))
citySalary_sum = citySalary.combineByKey(createCombiner,mergeValue,mergeCombiners)
citySalary_mean = citySalary_sum.map(lambda x:(x[0],float(x[1][0])/x[1][1]/10000,x[1][1]))
citySalary_mean = citySalary_mean.map(lambda x: (x[0],round((x[1]),1),x[2])).sortBy(lambda x : x[0])
citySalary_mean.take(1)
citySalary_mean = citySalary_mean.map(lambda x:{"city":x[0],"salary":x[1],"value":x[2]})
open('/home/hadoop/Desktop/data/citySalary.json','w').write(str(citySalary_mean.collect()))

9458

In [37]:
keywordNum = data_fields.map(lambda x : (x[1],1)).reduceByKey(lambda x,y : x+y)
keywordNum = keywordNum.map(lambda x:{"keyword":x[0],"value":x[1]})
open('/home/hadoop/Desktop/data/keywordNum.json','w').write(str(keywordNum.collect()))

4032